# Generation tutorial

This tutorial shows you how to generate slabs of different thicknesses, terminations and Miller indices systematically. It is commonly required to run a series of calculations on a set of slabs for one material to answer the questions: 

- How thick should my slab be to ensure it is bulk-like in the centre? 
- How much vacuum do I need to ensure one side of the slab does not interact with the other? (this is particularly useful for codes like VASP which enforce 3D periodic boundary conditions)
- Which Miller index and termination combinations have zero net dipole?
- Which Miller index and termination combinations have the lowest surface energy? 

The `surfaxe.generation` module allows for easy generation of zero-dipole, symmetric slabs for convergence testing purposes either up to a maximum Miller index or for a specific Miller index. 

By passing the structure, required Miller index, and lists of slab and vacuum thicknesses, `get_all_slabs` and `get_one_hkl_slabs` can generate unique zero-dipole symmetric surface slabs for all combinations of slab and vacuum thickness. As ever, [pymatgen](https://pymatgen.org) does most of the heavy lifting, but we have extra options to:
 
- Detect duplicated slabs
- Automatically save structures in a sensible file format
- Additionally save other DFT calculation input files so each slab is ready to run
- Output the directory structure in such a way that once the DFT calculations are run, `surfaxe.convergence` can easily extract the key data and plot it

In [1]:
# Import from relevant surfaxe modules
from surfaxe.generation import get_slabs_max_index, get_slabs_single_hkl

# Misc imports
from pathlib import Path
import os

# Ignore some specific benign warnings 
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", message="POTCAR data")
warnings.filterwarnings("ignore", message="Overriding the POTCAR functional")
warnings.filterwarnings("ignore", message=r".*is a deprecated alias for the builtin.*")
warnings.filterwarnings("ignore", message=r".*should_run_async.*")

In [2]:
# Set path to example data 
path_to_gener_data = Path.cwd().parents[1].joinpath('example_data/generation')

## Generate slabs for a specified Miller index
This simple example uses the output structure from a bulk relaxation of Y2Ti2S2O5. We create a series of slabs with thickness of 20 and 30 Angstroms, and vacuum thicknesses of 20, 30, 40 and 50 angstroms in the (0,0,1) direction. We suppress the default behaviour of saving input files by setting `save_slabs=False`.

In [6]:
slabs_001 = get_slabs_single_hkl(
                    structure=os.path.join(path_to_gener_data,'CONTCAR_conventional'),
                    hkl=(0,0,1), 
                    thicknesses=[20,30,40], vacuums=[20,30,40,50], 
                    save_slabs=False)

print('{} slabs generated'.format(len(slabs_001)))

6 slabs generated


If you leave `thicknesses=[20,30,40], vacuums=[20,30,40,50]` you will get a warning about repeted slabs that have been removed from the final list. We can inspect one entry of the list which is a dictionary, containing details and the slab itself as a pymatgen structure:

In [12]:
print(slabs_001[0])

{'hkl': '001', 'slab_t': 20, 'vac_t': 20, 's_index': 4, 'slab': Structure Summary
Lattice
    abc : 3.7700229987365135 3.7700229987365135 45.407026909472286
 angles : 90.0 90.0 90.0
 volume : 645.3734068396844
      A : 3.7700229987365135 0.0 2.308473299057289e-16
      B : -2.308473299057289e-16 3.7700229987365135 2.308473299057289e-16
      C : 0.0 0.0 45.407026909472286
PeriodicSite: Y3+ (1.8850, 1.8850, 20.8243) [0.5000, 0.5000, 0.4586]
PeriodicSite: Y3+ (0.0000, 0.0000, 1.8792) [0.0000, 0.0000, 0.0414]
PeriodicSite: Y3+ (0.0000, 0.0000, 9.4726) [0.0000, 0.0000, 0.2086]
PeriodicSite: Y3+ (1.8850, 1.8850, 13.2309) [0.5000, 0.5000, 0.2914]
PeriodicSite: Ti4+ (1.8850, 1.8850, 3.8570) [0.5000, 0.5000, 0.0849]
PeriodicSite: Ti4+ (0.0000, 0.0000, 18.8465) [0.0000, 0.0000, 0.4151]
PeriodicSite: Ti4+ (0.0000, 0.0000, 15.2088) [0.0000, 0.0000, 0.3349]
PeriodicSite: Ti4+ (1.8850, 1.8850, 7.4947) [0.5000, 0.5000, 0.1651]
PeriodicSite: S2- (1.8850, 1.8850, 1.0037) [0.5000, 0.5000, 0.0221]
Peri

Note: `s_index` is the position of the slab in the list of provisional slabs generated for each vacuum-slab thickness combo by the [underlying pymatgen function](https://pymatgen.org/pymatgen.core.surface.html#pymatgen.core.surface.SlabGenerator) before surfaxe gets rid of those with a dipole. The list is ordered by number of bonds broken to form the surface. It may be useful for book-keeping and debugging purposes, if you want to use the pymatgen function directly to more fully explore possible surface terminations of your material.

## Generate all slabs up to a maximum Miller index

We can generate all unique, zero-dipole, symmetric slabs up to a max Miller index of 1, this time for a smaller selection of slab and vacuum thicknesses: 

<div class="alert alert-info">

**Note:** If you are running this on Binder, you may want to reduce the number of vacuum and slab thicknesses to try for it to finish in a reasonable time. Try `thicknesses=[20,40], vacuums=[20]`.

</div>

In [16]:
get_slabs_max_index(
    structure=os.path.join(path_to_gener_data,'CONTCAR_conventional'),
                    max_index=1, 
                    thicknesses=[20,30], 
                    vacuums=[20,30])

By default `save_slabs=True`, so the structure files are saved to files. A directory has been created locally called `Y4Ti4S4O10` and each structure has been named in the format `POSCAR_miller-index_slab-thickness_vacuum-thickness_s-index`. The VASP POSCAR file is the default, but any other input file supported by pymatgen can be supplied as the `fmt` argument.  

### Make separate folders

We can get the function to organise these into directories by setting `make_fols=True`. The structure files will go into subfolders with the following structure: `hkl/slab_vacuum_index/POSCAR` in the `Y4Ti4S4O10` directory. 

In [3]:
get_slabs_max_index(
    structure=os.path.join(path_to_gener_data,'CONTCAR_conventional'),
                    max_index=1, 
                    thicknesses=[20,30], 
                    vacuums=[20,30], make_fols=True)

### Make all input files
This is currently only supported for VASP input files.

For generation of input files using `make_input_files=True`, reasonable INCAR tags, default POTCARS and KPOINTS are set according to preset configuration dictionaries (you can view them in the `suraxe/_config_dictionaries` directory). Further customisation can be done either via `user_incar_settings`, `user_potcar_settings` and `user_kpoints_settings` arguments or by adding custom config scripts. See the relevant docs of the underlying [pymaten module](https://pymatgen.org/pymatgen.io.vasp.sets.html). 

<div class="alert alert-info">

**Note:** This example requires that you have [set up your pymatgen POTCAR environment](https://pymatgen.org/installation.html#potcar-setup). It will not run on Binder or without first setting up your POTCAR directory. 

</div>

In [8]:
get_slabs_max_index(
    structure=os.path.join(path_to_gener_data,'CONTCAR_conventional'),
                    max_index=1, 
                    thicknesses=[20,30], 
                    vacuums=[20,30], make_input_files=True)

Notes:

- Depending on the number of slab and vacuum thicknesses, the max hkl specified and the complexity of the system the `get_all_unique_slabs` script may get slow. 

- If `make_fols` is set to `False`, but `make_input_files` is `True`, the function assumes behaviour as if `make_fols=True`. 

- For both generation scripts, UserWarnings are raised if there are repeat slabs and if the slabs are greater than the specified maximum size (default is 500 atoms). If a slab has a number of atoms greater than the `max_size`, it will be written out but a warning will be raised. 

- Oxidation states are added to the bulk structure by guess by default but can be added by element or by site as well.

- The slabs are centered by default - this makes the addition of adsorbates easier. The slabs are orthogonalised by default using the LLL reduction algorithm. For full customisabilty, all pymatgen `SlabGenerator`,  `generate_all_slabs` and `DictSet` keyword arguments are supported but not fully documented here.  

- By default surfaxe searches for slabs with Laue (inversion) symmetry. Such slabs cannot be cleaved from non-centrosymmetric bulk structures as they do not contain inversion symmetry. Surfaxe will automatically detect if the bulk structure is non-centrosymmetric and cleave zero-dipole slabs, regardless of their space group. The `is_symmetric` tag can be used with centrosymmetric structures to obtain all non-polar slabs, not just the symmetric ones. 